# Hypotheek, alles aflossen of niet?

Aannames
* hyptotheekrente aftrek regeling blijft zoals het nu is
* hyptotheekrente aftrek wordt geinvesteerd als het terug gegeven wordt
* het maandelijkse verschil tussen *X% aflosvrije hyptoheek* en *0% aflosvrije hyptotheek* wordt geinvesteerd
* de investerenen worden gedaan in een indexfund (globale aandelenmarkt)
* WOZ waarde stijgt net zo hard als huizenprijs

Variable
* aflossings vrij gedeelte in %
* rente over aflossingsvrij deel (*zonder* hyptotheekrente aftrek) en aflossingsdeel
* huizenprijs groei per jaar
* aandelenmarkt groei per jaar



In [166]:
from maandlasten import maandlasten

from mortgage import Mortgage

from inflation import inflatie_laatste_jaar

HOOGSTE_SCHIJF = 0.495  # belasting 49.5%


def per_maand(x):
    return (1 + x / 100) ** (1 / 12)


def run_hypotheek(
    aflossings_vrij=50,  # pct
    rente_deel_1=1.89,
    rente_deel_2=1.99,
    huis_groei_per_jaar=4,
    aandelen_groei_per_jaar=7,
    huisprijs=650e3,
):
    m1 = Mortgage(rente_deel_1 / 100, 360, huisprijs * (1 - aflossings_vrij / 100))
    m2 = huisprijs * aflossings_vrij / 100
    m2_rente = m2 * rente_deel_2 / 100 / 12

    # Benchmark is hyptotheek die 100% wordt afgelost
    m_benchmark = Mortgage(rente_deel_1 / 100, 360, huisprijs)

    def chunks(lst, n):
        for i in range(0, len(lst), n):
            yield lst[i : i + n]

    aftrek_per_jaar = []
    afbetaling, m1_rente = zip(*m1.monthly_payment_schedule())
    rente_per_jaar = [float(sum(x)) for x in chunks(m1_rente, 12)]
    woz = huisprijs * 0.95  # start op vorig jaar
    assert len(rente_per_jaar) == 30  # 30 jaar
    for m1_rente in rente_per_jaar:
        woz *= 1 + huis_groei_per_jaar / 100
        aftrek = maandlasten.hypotheek_aftrek(m1_rente, woz_waarde=woz)
        aftrek_per_jaar.append(aftrek)

    _groei_per_maand = (1 + aandelen_groei_per_jaar / 100) ** (1 / 12)
    totaal_betaald = 0
    invested = 0
    totaal_schuld = huisprijs
    huis_waarde = huisprijs
    benchmark_betaling = float(m_benchmark.monthly_payment())
    for i, (afbetaling, m1_rente) in enumerate(m1.monthly_payment_schedule()):
        if i % 12 == 0:
            # Hypotheekrente aftrek terug, dat investeren we ook
            invested += HOOGSTE_SCHIJF * aftrek_per_jaar[i // 12]

        # Vermogen groei
        huis_waarde *= per_maand(huis_groei_per_jaar)
        invested *= per_maand(aandelen_groei_per_jaar)

        # Verrekening
        te_betalen = float(afbetaling) + float(m1_rente) + m2_rente
        to_invest = benchmark_betaling - te_betalen
        totaal_betaald += te_betalen
        totaal_schuld -= float(afbetaling)
        invested += to_invest

    vermogen = invested + huis_waarde - totaal_schuld
    return {
        "vermogen": round(vermogen, 2),
        "schuld": round(totaal_schuld, 2),
        "betaald": round(totaal_betaald, 2),
        "huis_waarde": round(huis_waarde, 2),
        "invested": round(invested, 2),
    }


kw = dict(
    huis_groei_per_jaar=3,#inflatie_laatste_jaar(30),
    aandelen_groei_per_jaar=7,
    huisprijs=650_000,
)
h1 = run_hypotheek(aflossings_vrij=0, **kw)
h2 = run_hypotheek(aflossings_vrij=50, **kw)
verschil_vermogen = h2["vermogen"] - h1["vermogen"]
print(
    f"Het verschil door niet af te lossen is €{verschil_vermogen/1000:.0f}k (€{h1['vermogen']/1e6:.2f}M vs €{h2['vermogen']/1e6:.2f}M vermogen)."
    "\n"
    f"Het aandelenportfolio is €{h1['invested']/1000:.0f}k vs €{h2['invested']/1000:.0f}k."
    "\n"
    f"De uiteindelijke huiswaarde is €{h1['huis_waarde']/1e6:.2f}M."
)

Het verschil door niet af te lossen is €240k (€1.82M vs €2.06M vermogen).
Het aandelenportfolio is €241k vs €806k.
De uiteindelijke huiswaarde is €1.58M.


# Aandelen groei, break even point

In [161]:
from scipy.optimize import minimize_scalar


def break_even(groei):
    kw = dict(aandelen_groei_per_jaar=groei, huis_groei_per_jaar=4)
    h1 = run_hypotheek(aflossings_vrij=0, **kw)
    h2 = run_hypotheek(aflossings_vrij=50, **kw)
    return abs(h1["vermogen"] - h2["vermogen"])


min_aandelen_groei = minimize_scalar(break_even).x
f"Als de aandelen harder groeien dan {min_aandelen_groei:.2f}% per jaar, dan is aflosvrij beter!"

'Als de aandelen harder groeien dan 3.47% per jaar, dan is aflosvrij beter!'

In [143]:
h1, h2

({'vermogen': 2196869.09,
  'schuld': 0.0,
  'betaald': 852094.57,
  'huis_waarde': 2108208.38,
  'invested': 88660.71},
 {'vermogen': 2179383.77,
  'schuld': 325000.0,
  'betaald': 620071.75,
  'huis_waarde': 2108208.38,
  'invested': 396175.39})

In [101]:
koopkracht_factor = (1 + inflatie_laatste_jaar(30) / 100)**30

In [118]:
koopkracht_factor

1.7914016201837375

In [90]:
250/360

0.6944444444444444